In [1]:
/**
 The following notebook does:
     - Reads multi single- or a multi- band GeoTiff from HDFS
     - Filters out or converts NaN values
     - Runs Kmeans
     - Saves the kmeans model to HDFS
**/


import geotrellis.raster.MultibandTile
import geotrellis.spark.io.hadoop._
import geotrellis.vector.ProjectedExtent
import org.apache.hadoop.fs.Path
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.rdd.RDD

In [19]:
//The following example shows how to get the number of bands, however, it is not known 
//val band_count = geotrellis.raster.io.geotiff.reader.TiffTagsReader.read(filepath).bandCount;

val band_count = 1;
var band_RDD :RDD[Array[Double]] = sc.emptyRDD
val pattern: String = "tif"
var filepath :String = ""
if (band_count == 1) {
    //Single band GeoTiff
    filepath = "hdfs:///user/hadoop/spring-index/LastFreeze/"
} else {
    //Multi band GeoTiff
    filepath = "hdfs:///user/hadoop/spring-index/BloomFinal/"
}
    
if (band_count == 1) {
    //Lets load a Singleband GeoTiff and return RDD just with the tiles.
    //Since it is a single GeoTiff, it will be a RDD with a tile.
    val bands_RDD = sc.hadoopGeoTiffRDD(filepath, pattern).values
    
    //Lets filter out NaN
    band_RDD = bands_RDD.map(m => m.toArrayDouble().filter(!_.isNaN).take(1000000))
} else {
    //Lets load a Multiband GeoTiff and return RDD just with the tiles.
    //Since it is a single GeoTiff, it will be a RDD with a tile.
    val bands_RDD = sc.hadoopMultibandGeoTiffRDD(filepath, pattern).values
    
    //Extract the 4th band and filter out NaN
    band_RDD = bands_RDD.map(m => m.band(3).toArrayDouble().filter(v => !v.isNaN))
}

In [20]:
// Create a Vector without NaN values
val band_vec = band_RDD.map(s => Vectors.dense(s)).cache()

In [21]:
val numClusters = 3
val numIterations = 20
val clusters = {
    KMeans.train(band_vec,numClusters,numIterations)
}

// Evaluate clustering by computing Within Set Sum of Squared Errors
val WSSSE = clusters.computeCost(band_vec)
println("Within Set Sum of Squared Errors = " + WSSSE)

[Stage 13:======================================================> (35 + 1) / 36]Within Set Sum of Squared Errors = 5.975771808460358E9


In [22]:
//Un-persist the model
band_vec.unpersist()

// Shows the result.
println("Cluster Centers: ")
//clusters.clusterCenters.foreach(println)

//Clusters save the model
if (band_count == 1) {
    clusters.save(sc, "hdfs:///user/emma/spring_index/LastFreeze/all_kmeans_model")    
} else {
    clusters.save(sc, "hdfs:///user/emma/spring_index/BloomFinal/allkmeans_model")
}


Cluster Centers: 
[Stage 18:=============================>                            (2 + 2) / 4]